# _Linear Regression Lab 5 — Kevin Wong_#

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style(style="whitegrid")

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import summary_table
import scipy.stats as stats
from scipy.stats import t as tdist

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

/Users/kevinwong2014/anaconda/envs/py36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


** Dataset Context **

In the computational section of this Lab you will consider the baseball dataset found in
the file hitters.csv. This dataset records the salary of  n = 263 Major League Baseball
players during the 1987 season as well as q = 19 statistics associated with the
performance of each player during the previous season. Specifically, the dataset contains
observations from the following variables:
- AtBat: Number of times at bat in 1986
- Hits: Number of hits in 1986
- HmRun: Number of home runs in 1986
- Runs: Number of runs in 1986
- RBI: Number of runs batted in in 1986
- Walks: Number of walks in 1986
- Years: Number of years in the major leagues
- CAtBat: Number of times at bat during his career
- CHits: Number of hits during his career
- CHmRun: Number of home runs during his career
- CRuns: Number of runs during his career
- CRBI: Number of runs batted in during his career
- CWalks: Number of walks during his career
- League: A categorical variable with levels A (for American) and N (for National) indicating the player’s league at the end of 1986
- Division: A factor with levels E (for East) and W (for West) indicating the player’s division at the end of 1986
- PutOuts: Number of put outs in 1986
- Assists: Number of assists in 1986
- Errors: Number of errors in 1986
- Salary: 1987 annual salary on opening day in thousands of dollars
- NewLeague: A factor with levels A and N indicating the player’s league at the beginning of 1987

**Importing Data in Python**

In [2]:
df = pd.read_csv("hitters.csv") 
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 20 columns):
AtBat        263 non-null int64
Hits         263 non-null int64
HmRun        263 non-null int64
Runs         263 non-null int64
RBI          263 non-null int64
Walks        263 non-null int64
Years        263 non-null int64
CAtBat       263 non-null int64
CHits        263 non-null int64
CHmRun       263 non-null int64
CRuns        263 non-null int64
CRBI         263 non-null int64
CWalks       263 non-null int64
League       263 non-null object
Division     263 non-null object
PutOuts      263 non-null int64
Assists      263 non-null int64
Errors       263 non-null int64
Salary       263 non-null float64
NewLeague    263 non-null object
dtypes: float64(1), int64(16), object(3)
memory usage: 85.9 KB


In [3]:
df.head(10)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.000,N
1,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.000,A
2,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.000,N
3,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.500,N
4,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.000,A
5,185,37,1,23,8,21,2,214,42,1,30,9,24,N,E,76,127,7,70.000,A
6,298,73,0,24,24,7,3,509,108,0,41,37,12,A,W,121,283,9,100.000,A
7,323,81,6,26,32,8,2,341,86,6,32,34,8,N,W,143,290,19,75.000,N
8,401,92,17,49,66,65,13,5206,1332,253,784,890,866,A,E,0,0,0,1100.000,A
9,574,159,21,107,75,59,10,4631,1300,90,702,504,488,A,E,238,445,22,517.143,A


---
** A) Calculate the variance inflation factor (VIF) for each of the explanatory variables.
Comment on whether multicollinearity appears to be an issue. If it is, identify the
three explanatory variables that are most seriously affected by the issue. **

In [4]:
m_atbat = smf.ols("AtBat ~ Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_atbat = 1 / (1 - m_atbat.rsquared)
print("VIF atbat: ", VIF_atbat)

m_hits = smf.ols("Hits ~ AtBat + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_hits = 1 / (1 - m_hits.rsquared)
print("VIF hits: ", VIF_hits)

m_hmrun = smf.ols("Hits ~ AtBat + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_hmrun = 1 / (1 - m_hmrun.rsquared)
print("VIF HmRun: ", VIF_hmrun)

m_runs = smf.ols("Runs ~ AtBat + Hits + HmRun + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_runs = 1 / (1 - m_runs.rsquared)
print("VIF hits: ", VIF_runs)

m_rbi = smf.ols("RBI ~ AtBat + Hits + HmRun + Runs + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_rbi = 1 / (1 - m_rbi.rsquared)
print("VIF RBI: ", VIF_rbi)

m_walks = smf.ols("Walks ~ AtBat + Hits + HmRun + Runs + RBI + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_walks = 1 / (1 - m_walks.rsquared)
print("VIF Walks: ", VIF_walks)

m_years = smf.ols("Years ~ AtBat + Hits + HmRun + Runs + RBI + Walks + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_years = 1 / (1 - m_years.rsquared)
print("VIF Years: ", VIF_years)

m_catbat = smf.ols("CAtBat ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_catbat = 1 / (1 - m_catbat.rsquared)
print("VIF CAtBat: ", VIF_catbat)

m_chits = smf.ols("CHits ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_chits = 1 / (1 - m_chits.rsquared)
print("VIF CHits: ", VIF_chits)

m_chmrun = smf.ols("CHmRun ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_chmrun = 1 / (1 - m_chmrun.rsquared)
print("VIF CHmRun: ", VIF_chmrun)

m_cruns = smf.ols("CRuns ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_cruns = 1 / (1 - m_cruns.rsquared)
print("VIF CRuns: ", VIF_cruns)

m_crbi = smf.ols("CRBI ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_crbi = 1 / (1 - m_crbi.rsquared)
print("VIF CRBI: ", VIF_crbi)

m_cwalks = smf.ols("CWalks ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_cwalks = 1 / (1 - m_cwalks.rsquared)
print("VIF CWalks: ", VIF_cwalks)

m_putouts = smf.ols("PutOuts ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + Assists + Errors + C(NewLeague)", data = df).fit()
VIF_putouts = 1 / (1 - m_putouts.rsquared)
print("VIF PutOuts: ", VIF_putouts)

m_assists = smf.ols("Assists ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Errors + C(NewLeague)", data = df).fit()
VIF_assists = 1 / (1 - m_assists.rsquared)
print("VIF Assists: ", VIF_assists)

m_errors = smf.ols("Errors ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + C(NewLeague)", data = df).fit()
VIF_errors = 1 / (1 - m_errors.rsquared)
print("VIF Errors: ", VIF_errors)

VIF atbat:  22.9443659168
VIF hits:  30.2812553305
VIF HmRun:  30.2812553305
VIF hits:  15.2464175021
VIF RBI:  11.9217150823
VIF Walks:  4.14871197167
VIF Years:  9.31327990234
VIF CAtBat:  251.561159565
VIF CHits:  502.954289038
VIF CHmRun:  46.4884615297
VIF CRuns:  162.52081015
VIF CRBI:  131.965857676
VIF CWalks:  19.7441050138
VIF PutOuts:  1.23631696225
VIF Assists:  2.70934093676
VIF Errors:  2.21454346669


Clearly, there does appear to be an issue of multicollinearity between several columns in the dataset.  The most significant variables with the highest VIFs are CHits, CAtBat, and CRuns.   This makes sense, as these are all career length metrics that track in essence how much someone has played.   

---
** Parts B) - E) completed using R specific libraries — R code attached.  **
<n>** Algorithm explanations: **

<n><i> All-Possible-Subsets approach</i>:  This method involves fitting all possible combinations of models uisng the explanatory variables and choosing the best one using evaluation metrics that penalize for increased numbers of explanatory terms (e.g. R-adj, BIC, AIC).  We fit $2^q$ models in total, choosing the best model within each "bucket" of models with a certain number of explanatory variables.  We then compare the best of each bucket and choose the one with the best goodness-of-fit metric, aka the optimized model.   

<n><i> Forward Selection</i>: This variant of stepwise selection begins with the smallest model — the intercept only model — and iterates forward by considering adding one more explanatory variable if it improves on a goodness-of-fit metric.  We continue to add explanatory variables one at a time until no further improvements can be made. 

<n><i> Backward Selection</i>: This variant of stepwise selection is like forward selection, only in reverse.  It starts with the bigest model and considers dropping the least influential explanatory variable, seeing if the goodness-of-fit metric can be improved by dropping it.  If it can, we drop the variable and then consider dropping one of the variables that remain until no improvements can be made.

<n><i> Hybrid Selection</i>: A combination of the previous two approaches, it starts with the bare intercept-only model and proceeds forward by considering to add one explanatory variable.  If adding an explanatory variable improves the metric, we add it.  Then, at each subsequent stage, we consider adding the most influential variable or eliminating the least influential variable — whichever improves the model the most.  We continue until no further improvements can be made.



---
** F) In this part you will compare the predictive performance of four models: **
- The full model with all 19 explanatory variables.
- The optimal model identified in part (b).
- The best model from parts (c)-(e) (i.e., the best stepwise-selection model).
- The model that is considered optimal with respect to the Bayesian Information Criterion (BIC) which contains the variables AtBat, Hits, Walks, CRBI, Division and PutOuts. 

Randomly split the observed data into a training set (containing roughly 80% of
all of the data) and a held-out test set (containing roughly 20% of all of the data).
Calculate the predictive root-mean-square error (RMSE) for each of the four
models. Which model appears to be most appropriate? Justify why this model is
most appropriate.

In [5]:
# the full model
m_full = smf.ols("Salary ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists + Errors + C(NewLeague)", data = df).fit()
X_full = df[["AtBat","Hits","HmRun","Runs","RBI","Walks","Years","CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks","League","Division","PutOuts","Assists","Errors","NewLeague"]]
y = df["Salary"]

X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size = 0.2)
train = pd.concat([X_train, y_train], axis = 1)
pred = m_full.predict(X_test)
RMSE_full = np.sqrt(np.mean((y_test - pred)**2))
print(RMSE_full)

398.187775948


In [6]:
# from results in R we find the:
# optimal model with 11 explanatory variables
m_optimal = smf.ols("Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + C(League) + C(Division) + PutOuts + Assists", data = df).fit()
X_optimal = df[["AtBat","Hits","Walks","CAtBat","CRuns","CRBI","CWalks","League","Division","PutOuts","Assists"]]

X_train, X_test, y_train, y_test = train_test_split(X_optimal, y, test_size = 0.2)
train = pd.concat([X_train, y_train], axis = 1)
pred = m_optimal.predict(X_test)
RMSE_optimal = np.sqrt(np.mean((y_test - pred)**2))
print(RMSE_optimal)

289.705274268


In [7]:
# the best stepwise selection model 
m_stepwise = smf.ols("Salary ~ CRBI + Hits + PutOuts + C(Division) + AtBat +  Walks + CWalks + CRuns + CAtBat + Assists", data = df).fit()
X_stepwise = df[["CRBI","Hits","PutOuts","Division","AtBat","Walks","CWalks","CRuns","CAtBat","Assists"]]

X_train, X_test, y_train, y_test = train_test_split(X_stepwise, y, test_size = 0.2)
train = pd.concat([X_train, y_train], axis = 1)
pred = m_stepwise.predict(X_test)
RMSE_stepwise = np.sqrt(np.mean((y_test - pred)**2))
print(RMSE_stepwise)

318.264723417


In [8]:
# the model considered optimal w.r.t. the Bayesian Information Criterion 
m_bic = smf.ols("Salary ~ AtBat +  Hits + Walks + CRBI + C(Division) + PutOuts", data = df).fit()
X_bic = df[["AtBat","Hits","Walks","CRBI","Division","PutOuts"]]

X_train, X_test, y_train, y_test = train_test_split(X_bic, y, test_size = 0.2)
train = pd.concat([X_train, y_train], axis = 1)
pred = m_bic.predict(X_test)
RMSE_bic = np.sqrt(np.mean((y_test - pred)**2))
print(RMSE_bic)

290.602586725


Based purely off of one permutation of data into training and testing sets, we see that the RMSE of all the models all are in roughly the same 200-400 range.  Specifically, the root mean squared error of the optimal model from part B is lowest, so it appears to be the most appropriate.  Moreover, the methodology of this approach is the most complete when considered from a goodness-of-fit perspective, as we have optimized for an accuracy metric that penalizes more terms.  Ideally, we want the simplest, most powerful model.  The stepwise model is just a faster approximation of the model from the optimal all-possible-subsets approach.  

---
**G) As in part F, you must compare the predictive performance of the same four
models, but here you must determine the predictive accuracy (predictive RMSE)
by using 10-Fold Cross Validation. Which model appears to be most appropriate?
Justify why this model is most appropriate. ** 

In [9]:
## K-fold Cross Validation on the full model
numfolds = 10

kf = KFold(n_splits=10, shuffle = True)
MSE = 0
for train_index, test_index in kf.split(X_full):
    train_X, test_X = X_full.loc[train_index], X_full.loc[test_index]
    train_y, test_y = y[train_index], y[test_index]
    training = pd.concat([train_X, train_y], axis = 1)
    pred = m_full.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE_full = np.sqrt(MSE/numfolds)
RMSE_full

303.0815240701595

In [10]:
## K-fold Cross Validation on the optimal model
numfolds = 10

kf = KFold(n_splits=10, shuffle = True)
MSE = 0
for train_index, test_index in kf.split(X_optimal):
    train_X, test_X = X_optimal.loc[train_index], X_optimal.loc[test_index]
    train_y, test_y = y[train_index], y[test_index]
    training = pd.concat([train_X, train_y], axis = 1)
    pred = m_optimal.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE_optimal = np.sqrt(MSE/numfolds)
RMSE_optimal

304.29513119942254

In [11]:
## K-fold Cross Validation on the stepwise model
numfolds = 10

kf = KFold(n_splits=10, shuffle = True)
MSE = 0
for train_index, test_index in kf.split(X_stepwise):
    train_X, test_X = X_stepwise.loc[train_index], X_stepwise.loc[test_index]
    train_y, test_y = y[train_index], y[test_index]
    training = pd.concat([train_X, train_y], axis = 1)
    pred = m_stepwise.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE_stepwise = np.sqrt(MSE/numfolds)
RMSE_stepwise

304.84833646250286

In [12]:
## K-fold Cross Validation on the BIC model
numfolds = 10

kf = KFold(n_splits=10, shuffle = True)
MSE = 0
for train_index, test_index in kf.split(X_bic):
    train_X, test_X = X_bic.loc[train_index], X_bic.loc[test_index]
    train_y, test_y = y[train_index], y[test_index]
    training = pd.concat([train_X, train_y], axis = 1)
    pred = m_bic.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE_bic = np.sqrt(MSE/numfolds)
RMSE_bic

315.20901854824081

From the k-fold RMSE calculations, it is hard to justify any one model as the singularly best as the first three are all so close in range.  Due to methodology, we know the optimal model will be as good or better than the stepwise model.  Considering the full model versus the optimal model, using BIC metrics, the optimal model is better.  However, looking at predictive RMSE, they are roughly equal.  

---
**H) Given the estimates of predictive accuracy from parts F and G indicate which
estimates you believe to be more accurate. In other words, indicate which
validation approach (i.e., cross validation vs. k-fold cross validation) you believe
will most accurately estimate the predictive capability of a model. Briefly explain
your rationale.**

K-fold cross validation gives a more accurate estimate of the predictive capability of the model.  Rather than splitting the data just once into a training and testing set and seeing how the model performs on the test set, we effectively do this process k different times in k-fold cross validation, testing the k models on different test sets.  We therefore get a more average estimate of how the model performs, which is less biased than normal cross validation.  

---
** I) Accounting for all of the analyses you’ve performed (i.e., multicollinearity,
goodness-of-fit, and predictive accuracy), which model would you be most
comfortable using? Briefly justify your choice.**

In [13]:
m_full.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                     15.39
Date:                Sun, 25 Nov 2018   Prob (F-statistic):           7.84e-32
Time:                        23:10:14   Log-Likelihood:                -1876.2
No. Observations:                 263   AIC:                             3792.
Df Residuals:                     243   BIC:                             3864.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           163.1036     90.779      1.797      0.074     -15.710     341.917
C(League)[T.N]       62.5994     79.261      0.790      0.430     -93.528     218.727
C(Division)[T.W]   -116.8492     40.367     -2.895      0.004    -196.363     -37.335
C(NewLeague)[T.N]   -24.7623     79.003     -0.313      0.754    -180.380     130.855
AtBat                -1.9799      0.634     -3.123      0.002      -3.229      -0.731
Hits                  7.5008      2.378      3.155      0.002       2.818      12.184
HmRun                 4.3309      6.201      0.698      0.486      -7.885      16.546
Runs                 -2.3762      2.981     -0.797      0.426      -8.248       3.495
RBI                  -1.0450      2.601     -0.402      0.688      -6.168       4.078
Walks                 6.2313      1.829      3.408      0.001       2.630       9.833
Years                -3.4891     12.412     -0.281      0.779     -27.938      20.960
CAtBat               -0.1713      0.135     -1.267      0.206      -0.438       0.095
CHits                 0.1340      0.675      0.199      0.843      -1.195       1.463
CHmRun               -0.1729      1.617     -0.107      0.915      -3.358       3.013
CRuns                 1.4543      0.750      1.938      0.054      -0.024       2.933
CRBI                  0.8077      0.693      1.166      0.245      -0.557       2.172
CWalks               -0.8116      0.328     -2.474      0.014      -1.458      -0.165
PutOuts               0.2819      0.077      3.640      0.000       0.129       0.434
Assists               0.3711      0.221      1.678      0.095      -0.065       0.807
Errors               -3.3608      4.392     -0.765      0.445     -12.011       5.290
==============================================================================
Omnibus:                       87.414   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              452.923
Skew:                           1.236   Prob(JB):                     4.46e-99
Kurtosis:                       8.934   Cond. No.                     2.09e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.09e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
m_optimal.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.543
Model:                            OLS   Adj. R-squared:                  0.523
Method:                 Least Squares   F-statistic:                     27.07
Date:                Sun, 25 Nov 2018   Prob (F-statistic):           8.93e-37
Time:                        23:10:14   Log-Likelihood:                -1877.2
No. Observations:                 263   AIC:                             3778.
Df Residuals:                     251   BIC:                             3821.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          135.7512     71.346      1.903      0.058      -4.762     276.265
C(League)[T.N]      43.1116     39.966      1.079      0.282     -35.600     121.823
C(Division)[T.W]  -111.1460     39.218     -2.834      0.005    -188.385     -33.907
AtBat               -2.1277      0.537     -3.959      0.000      -3.186      -1.069
Hits                 6.9237      1.646      4.206      0.000       3.682      10.166
Walks                5.6203      1.591      3.533      0.000       2.488       8.753
CAtBat              -0.1390      0.056     -2.478      0.014      -0.249      -0.029
CRuns                1.4553      0.393      3.706      0.000       0.682       2.229
CRBI                 0.7853      0.210      3.743      0.000       0.372       1.198
CWalks              -0.8229      0.264     -3.121      0.002      -1.342      -0.304
PutOuts              0.2894      0.075      3.870      0.000       0.142       0.437
Assists              0.2688      0.158      1.700      0.090      -0.043       0.580
==============================================================================
Omnibus:                       88.563   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              470.516
Skew:                           1.246   Prob(JB):                    6.74e-103
Kurtosis:                       9.060   Cond. No.                     1.39e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [15]:
m_stepwise.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.540
Model:                            OLS   Adj. R-squared:                  0.522
Method:                 Least Squares   F-statistic:                     29.64
Date:                Sun, 25 Nov 2018   Prob (F-statistic):           2.80e-37
Time:                        23:10:14   Log-Likelihood:                -1877.8
No. Observations:                 263   AIC:                             3778.
Df Residuals:                     252   BIC:                             3817.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          162.5354     66.908      2.429      0.016      30.766     294.305
C(Division)[T.W]  -112.3801     39.214     -2.866      0.005    -189.610     -35.150
CRBI                 0.7743      0.210      3.694      0.000       0.362       1.187
Hits                 6.9180      1.647      4.201      0.000       3.675      10.161
PutOuts              0.2974      0.074      3.995      0.000       0.151       0.444
AtBat               -2.1687      0.536     -4.044      0.000      -3.225      -1.112
Walks                5.7732      1.585      3.643      0.000       2.652       8.894
CWalks              -0.8308      0.264     -3.152      0.002      -1.350      -0.312
CRuns                1.4082      0.390      3.607      0.000       0.639       2.177
CAtBat              -0.1301      0.055     -2.344      0.020      -0.239      -0.021
Assists              0.2832      0.158      1.796      0.074      -0.027       0.594
==============================================================================
Omnibus:                       91.407   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              492.766
Skew:                           1.288   Prob(JB):                    9.93e-108
Kurtosis:                       9.191   Cond. No.                     1.28e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.28e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
m_bic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Salary   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                     44.18
Date:                Sun, 25 Nov 2018   Prob (F-statistic):           6.82e-37
Time:                        23:10:14   Log-Likelihood:                -1886.6
No. Observations:                 263   AIC:                             3787.
Df Residuals:                     256   BIC:                             3812.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           91.5118     65.000      1.408      0.160     -36.491     219.515
C(Division)[T.W]  -122.9515     39.820     -3.088      0.002    -201.369     -44.534
AtBat               -1.8686      0.527     -3.543      0.000      -2.907      -0.830
Hits                 7.6044      1.663      4.574      0.000       4.330      10.878
Walks                3.6976      1.210      3.055      0.002       1.314       6.081
CRBI                 0.6430      0.064      9.979      0.000       0.516       0.770
PutOuts              0.2643      0.075      3.535      0.000       0.117       0.412
==============================================================================
Omnibus:                      101.319   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              615.052
Skew:                           1.411   Prob(JB):                    2.77e-134
Kurtosis:                       9.940   Cond. No.                     2.29e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.29e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Accounting for everything, the predictive RMSE scores suggest that the optimal or full model is best.  However, the multicollinearity test at the beginning showed that multicollinearity is a problem in our data, so we should be concerned about taking unimportant variables out of our model.  CHits is a variable with a very high VIF, which was taken out of the full model in the optimal model, along with other less important variables.  Thus, not only is the optimal model a simpler model that still has equivalent predictive power — it also has a marginally higher adjusted R-squared and explains a higher proportion of the variance.  Through a consideration of goodness-of-fit, predictive power, multicollinearity, and methodology we conclude that the all-possible-subsets optimal approach gives the best model.  